In [ ]:
# TEST 1: Variable Count

# ABSTRACT --------------------------------------------------------------------------------------------------------------------------------------------- 
- This test is to check if the BQM contains all required variables. Nothing extra, Nothing missing.

# METHOD ------------------------------------------------------------------------------------------------------------------------------------------------
- We can simply check that using the method: len(bqm.variables). Then compare it with the expected number, 
which can be obtained via the formula -> Expected = Nvehicles x Nlocations x (Nlocations−1). We will try
three different combinations and check the result. Additionally, the set costs will be replaced by an algorithm
that creates a dummy cost dictionary accordingly.

Combination 1: nodes = [0, 1, 2]     customers = [1, 2]     vehicles = ['a', 'b']
Expected Number: 12

Combination 2: nodes = [0, 1, 2, 3, 4, 5]     customers = [1, 2, 3, 4, 5]     vehicles = ['a']
Expected Number: 30

Combination 3: nodes = [0, 1, 2, 3, 4, 5, 6, 7, 8]     customers = [1, 2, 3, 4, 5, 6]     vehicles = ['a', 'b', 'c', 'd']
Expected Number: 288
    
# ADDED CODE -------------------------------------------------------------------------------------------------------------------------------------------
def GenerateDummyT(Nlocations, min_cost, max_cost, symmetric):
    t = {}
    for i, j in itertools.permutations(range(Nlocations), 2):
        if symmetric and (j, i) in t:
            t[(i, j)] = t[(j, i)]
        else:
            t[(i, j)] = random.randint(min_cost, max_cost)

    return t

print("Number of variables:", len(bqm.variables))

# RESULTS ----------------------------------------------------------------------------------------------------------------------------------------------
Combination 1: Correct Output
Combination 2: Correct Output
Combination 3: Correct Output

# CONCLUSION -------------------------------------------------------------------------------------------------------------------------------------------
- Therefore, the algorithm correctly creates the required variables.
    

In [35]:
import dimod
from dimod import ConstrainedQuadraticModel, Binary
from dimod import quicksum

nodes = [0, 1, 2, 3, 4, 5, 6, 7, 8]
customers = [1, 2, 3, 4, 5, 6, 7, 8]
vehicles = ['a', 'b', 'c', 'd']



#------TEST-------------------------------------------------------
import itertools
import random
def GenerateDummyT(Nlocations, min_cost, max_cost, symmetric):
    t = {}
    for i, j in itertools.permutations(range(Nlocations), 2):
        if symmetric and (j, i) in t:
            t[(i, j)] = t[(j, i)]
        else:
            t[(i, j)] = random.randint(min_cost, max_cost)

    return t
#-----------------------------------------------------------------

t = GenerateDummyT(9, 1, 5, True)

# Create binary decision variables                   
# ---------------------------------
x = {}
for i in nodes:
    for j in nodes:
        if i != j: 
            for k in vehicles:
                name = f"x_{i}_{j}_{k}"
                x[i,j,k] = Binary(name)


# Building the CQM
# -----------------
cqm = ConstrainedQuadraticModel()

# 1) Objective: minimize total travel time
objective = quicksum(t[i,j] * x[i,j,k]
                     for i,j,k in x.keys())
cqm.set_objective(objective)

# Constraints
# -------------

# Each customer visited once
for v in customers:
    cqm.add_constraint(
        quicksum(x[i,v,k] for i in nodes if i != v for k in vehicles) == 1,
        label=f"visit_{v}"
    )

# Start and end at Depot for each vehicle
for k in vehicles:
    # start from depot once
    cqm.add_constraint(
        quicksum(x[0,j,k] for j in customers) == 1,
        label=f"start_{k}"
    )
    # return to depot once
    cqm.add_constraint(
        quicksum(x[i,0,k] for i in customers) == 1,
        label=f"end_{k}"
    )

# Flow conservation
for v in customers:
    for k in vehicles:
        cqm.add_constraint(
            quicksum(x[i,v,k] for i in nodes if i != v) -
            quicksum(x[v,j,k] for j in nodes if j != v) == 0,
            label=f"flow_{v}_{k}"
        )

# Converting to BQM 
bqm, invert = dimod.cqm_to_bqm(cqm)


#------TEST-------------------------------------------------------
print("Number of variables:", len(bqm.variables))
#-----------------------------------------------------------------

CQM created with 48 constraints and 288 variables.
Number of variables: 288
